# Use custom software_spec to create statsmodels function describing data with `ibm-watsonx-ai`

This notebook demonstrates how to deploy in Watson Machine Learning service a python function with `statsmodel` which requires to create custom software specification using conda yaml file with all required libraries.  
Some familiarity with bash is helpful. This notebook uses Python 3.11 with statsmodel.

## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning instance
-  Creating custom software specification
-  Online deployment of python function
-  Scoring data using deployed function

## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Function creation](#create)
3. [Function upload](#upload) 
4. [Web service creation](#deploy)
5. [Scoring](#score)
6. [Clean up](#cleanup)
7. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pak for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="create"></a>
## 2. Create function

In this section you will learn how to create deployable function
with statsmodels module calculating describition of a given data.  
**Hint**: To install statsmodels execute `!pip install statsmodels`.

#### Create deploayable callable which uses stsmodels library

In [9]:
def deployable_callable():
    """
    Deployable python function with score
    function implemented.
    """
    try:
        from statsmodels.stats.descriptivestats import describe
    except ModuleNotFoundError as e:
        print(f"statsmodels not installed: {str(e)}")
        
    def score(payload):
        """
        Score method.
        """
        try:
            data = payload['input_data'][0]['values']
            return {
                'predictions': [
                    {'values': str(describe(data))}
                ]
            }
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
        
    return score

####  Test callable locally

**Hint**: To install numpy execute `!pip install numpy`.

In [10]:
import numpy as np

data = np.random.randn(10, 10)
data_description = deployable_callable()({
    "input_data": [{
        "values" : data
    }]
})

print(data_description["predictions"][0]["values"])

                          0          1          2          3          4  \
nobs              10.000000  10.000000  10.000000  10.000000  10.000000   
missing            0.000000   0.000000   0.000000   0.000000   0.000000   
mean               0.905005  -0.182868   0.212348  -0.196478  -0.066239   
std_err            0.298759   0.434762   0.329075   0.411152   0.338328   
upper_ci           1.490562   0.669249   0.857322   0.609365   0.596871   
lower_ci           0.319448  -1.034986  -0.432627  -1.002320  -0.729350   
std                0.944759   1.374838   1.040625   1.300175   1.069887   
iqr                1.096161   1.229678   1.127913   1.153995   1.466211   
iqr_normal         0.812586   0.911562   0.836123   0.855458   1.086904   
mad                0.708938   1.064078   0.810868   0.959040   0.869382   
mad_normal         0.888522   1.333624   1.016272   1.201979   1.089609   
coef_var           1.043927  -7.518190   4.900571  -6.617419 -16.151875   
range              3.0088

<a id="upload"></a>
## 3. Upload python function

In this section you will learn how to upload the python function to the Cloud.

#### Custom software_specification
Create new software specification based on default Python 3.11 environment extended by autoai-libs package.

In [14]:
config_yml =\
"""name: python311
channels:
  - empty
  - nodefaults
dependencies:
- pip:
  - statsmodels

prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [15]:
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")

In [16]:
!cat config.yaml

name: python311
channels:
  - empty
  - nodefaults
dependencies:
- pip:
  - statsmodels

prefix: /opt/anaconda3/envs/python311


`config.yaml` file describes details of package extention. Now you need to store new package extention with APIClient.

In [17]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "statsmodels env",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with statsmodels",
    client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


#### Create new software specification and add created package extention to it.

In [18]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "statsmodels software_spec",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for statsmodels",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)

client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


'SUCCESS'

#### Get the details of created software specification

In [ ]:
client.software_specifications.get_details(sw_spec_id)

#### Store the function

In [21]:
meta_props = {
    client.repository.FunctionMetaNames.NAME: "statsmodels function",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}

function_details = client.repository.store_function(meta_props=meta_props, function=deployable_callable)
function_id = client.repository.get_function_id(function_details)

#### Get function details

In [ ]:
client.repository.get_details(function_id)

**Note:** You can see that function is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_functions()

<a id="deploy"></a>
## 4. Create online deployment
You can use commands bellow to create online deployment for stored function (web service).

#### Create online deployment of a python function

In [23]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of statsmodels function",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment = client.deployments.create(function_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: 'fe302e1b-17d9-470d-9884-5d5d515e14ea' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....................
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='6828cd36-5548-4e67-b1b3-d67547af8630'
-----------------------------------------------------------------------------------------------




In [ ]:
client.deployments.list()

Get deployment id.

In [24]:
deployment_id = client.deployments.get_id(function_deployment)
print(deployment_id)

6828cd36-5548-4e67-b1b3-d67547af8630


<a id="score"></a>
## 5. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [25]:
scoring_payload = {
    "input_data": [{
        'values': data
    }]
}

In [26]:
predictions = client.deployments.score(deployment_id, scoring_payload)
print(data_description["predictions"][0]["values"])

                          0          1          2          3          4  \
nobs              10.000000  10.000000  10.000000  10.000000  10.000000   
missing            0.000000   0.000000   0.000000   0.000000   0.000000   
mean               0.905005  -0.182868   0.212348  -0.196478  -0.066239   
std_err            0.298759   0.434762   0.329075   0.411152   0.338328   
upper_ci           1.490562   0.669249   0.857322   0.609365   0.596871   
lower_ci           0.319448  -1.034986  -0.432627  -1.002320  -0.729350   
std                0.944759   1.374838   1.040625   1.300175   1.069887   
iqr                1.096161   1.229678   1.127913   1.153995   1.466211   
iqr_normal         0.812586   0.911562   0.836123   0.855458   1.086904   
mad                0.708938   1.064078   0.810868   0.959040   0.869382   
mad_normal         0.888522   1.333624   1.016272   1.201979   1.089609   
coef_var           1.043927  -7.518190   4.900571  -6.617419 -16.151875   
range              3.0088

<a id="cleanup"></a>
## 6. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for function deployment and scoring with custom software_spec.  

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Jan Sołtysik** Intern in Watson Machine Learning.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.